In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image

import sys
sys.path.append("../../../src/")
sys.path.append("../../")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

## Hyperparameter

In [3]:
# selected_layer = 1
# selected_filter = 47
# selected_layer = 3
# selected_filter = 20
# selected_layer = 6
# selected_filter = 19
# selected_layer = 13
# selected_filter = 112
# selected_layer = 15
# selected_filter = 148
# selected_layer = 20
# selected_filter = 398
selected_layer = 22
selected_filter = 485
# selected_layer = 25
# selected_filter = 1
# selected_layer = 8
# selected_filter = 99
# selected_layer = 18
# selected_filter = 101
# selected_layer = 11
# selected_filter = 75
num_class = 30
exp = "052517"
class_index = 950
num_class = 30

In [4]:
backbone = "vgg16"
resume = "037-0"
color_map = "nipy_spectral"
model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
import torchvision
net2 = torchvision.models.vgg16_bn().cuda()

In [25]:
summary(net2, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,

## Load modal from dict

In [5]:
# model
net = model.Network(backbone=backbone, num_classes=1000, 
                    selected_layer=selected_layer)
net.to(device)

# resume from model
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt, strict=False) 

Only Keep 22th layers before.
Resume from model from exp: 037 at epoch 0


_IncompatibleKeys(missing_keys=[], unexpected_keys=['model.fc.0.weight', 'model.fc.0.bias', 'model.fc.3.weight', 'model.fc.3.bias', 'model.fc.6.weight', 'model.fc.6.bias', 'model.features.24.weight', 'model.features.24.bias', 'model.features.26.weight', 'model.features.26.bias', 'model.features.28.weight', 'model.features.28.bias'])

In [6]:
summary(net, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

## Load Original Data 

In [7]:
# Load data
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
reverse_mean = [-0.485, -0.456, -0.406]
reverse_std = [1/0.229, 1/0.224, 1/0.225]
train_transform = None
train_transform = transforms.Compose([                                                                                                                                
    transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
    transforms.ToTensor(),                                                                                                                                            
    #transforms.Normalize(mean, std)                                                                                                                                   
       ])         
trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", 
                             is_train=True, transform=train_transform)

trainset.set_data([class_index], num_class)
imgs_path = []                                                                                                                                                            
images = []
labels = []
for img, label, img_path in trainset:                                                                                                                                     
    images.append(img.unsqueeze(0))                                                                                                                                       
    labels.append(label)                                                                                                                                                  
    imgs_path.append(img_path)  

Len of new dataset is :30


## Create some need path

In [8]:
save_dir = "../../../saved/pack/"
################### Hyper-Parameter #######################
# exp = resume_exp
# epoch = resume_epoch
##########################################################
ab_path = os.path.join(save_dir, exp)

In [9]:
from pathlib import Path

In [10]:
index2image = {index: item.split("/")[-1].split(".")[0] for index, item in enumerate(imgs_path)}
index2image

{0: 'n07747607_10061',
 1: 'n07747607_10068',
 2: 'n07747607_10110',
 3: 'n07747607_10134',
 4: 'n07747607_10162',
 5: 'n07747607_10166',
 6: 'n07747607_10178',
 7: 'n07747607_10179',
 8: 'n07747607_10184',
 9: 'n07747607_10192',
 10: 'n07747607_10204',
 11: 'n07747607_10205',
 12: 'n07747607_1021',
 13: 'n07747607_10213',
 14: 'n07747607_10217',
 15: 'n07747607_10232',
 16: 'n07747607_10258',
 17: 'n07747607_10290',
 18: 'n07747607_10305',
 19: 'n07747607_10345',
 20: 'n07747607_10358',
 21: 'n07747607_10379',
 22: 'n07747607_10420',
 23: 'n07747607_10425',
 24: 'n07747607_10436',
 25: 'n07747607_10465',
 26: 'n07747607_10482',
 27: 'n07747607_10485',
 28: 'n07747607_10491',
 29: 'n07747607_10545'}

In [11]:
images_cpu = np.array([image.detach().clone().cpu().numpy().squeeze() for image in images])

In [12]:
# Load image
# ext may be different.
optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True, ext=".png")
valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
optimized_data_zscore = zscore(optimized_data, mean, std)
images_zscore = zscore(images_cpu, mean, std)

Load from ../../../saved/pack/052517/299999/n07747607_10061.png
Load from ../../../saved/pack/052517/299999/n07747607_10068.png
Load from ../../../saved/pack/052517/299999/n07747607_10110.png
Load from ../../../saved/pack/052517/299999/n07747607_10134.png
Load from ../../../saved/pack/052517/299999/n07747607_10162.png
Load from ../../../saved/pack/052517/299999/n07747607_10166.png
Load from ../../../saved/pack/052517/299999/n07747607_10178.png
Load from ../../../saved/pack/052517/299999/n07747607_10179.png
Load from ../../../saved/pack/052517/299999/n07747607_10184.png
Load from ../../../saved/pack/052517/299999/n07747607_10192.png
Load from ../../../saved/pack/052517/299999/n07747607_10204.png
Load from ../../../saved/pack/052517/299999/n07747607_10205.png
Load from ../../../saved/pack/052517/299999/n07747607_1021.png
Load from ../../../saved/pack/052517/299999/n07747607_10213.png
Load from ../../../saved/pack/052517/299999/n07747607_10217.png
Load from ../../../saved/pack/052517/2999

In [13]:
# Move to device
opt_image = torch.from_numpy(optimized_data_zscore).to(device)
original_image = torch.from_numpy(images_zscore).to(device)

## Obtain feature from conv layers.

In [14]:
ori_activation_maps = net.get_activation_maps(original_image, 
                                              selected_layer=selected_layer)[0]
opt_activation_maps = net.get_activation_maps(opt_image, 
                                              selected_layer=selected_layer)[0]

=> Register fhook 22
=> Register fhook 22


In [15]:
images_cpu_p, optimized_data_cpu, ori_activation_maps_cpu, opt_activation_maps_cpu =\
    preprocess_arrays(images_cpu, optimized_data,
                      ori_activation_maps[:, selected_filter], 
                      opt_activation_maps[:, selected_filter])
concated_imgs = concat_imgs(images_cpu_p, optimized_data_cpu,
                            ori_activation_maps_cpu, opt_activation_maps_cpu)

## Visualization Here.

In [16]:
save_path = os.path.join("../../../saved/pack/", exp, 
                         "concat.png")
concated_imgs.save(save_path)